In [1]:
import pandas as pd
import numpy as np
import glob
import os
from scipy import stats

# Momentum strategy 1: Selecting top and bottom performing stocks for long and short positions in the portfolio based on month-end observations.

### Read ticker data downloaded from quantquote

In [2]:
files = glob.glob("/home/home/vbox_shared/projects/Momentum-Trading/daily/*.csv")
df = pd.concat([pd.read_csv(fp,names=['date','0','open','high','low','close','volume'])
                .assign(ticker=os.path.basename(fp).replace('_','.').split('.')[1]) for fp in files])
df['date'] = pd.to_datetime(df['date'],format='%Y%m%d')
print (df)


           date  0     open     high      low    close        volume ticker
0    1998-01-02  0  23.5573  23.6455  23.2714  23.5573  7.108452e+06     ko
1    1998-01-05  0  23.5784  23.6031  23.0279  23.4479  1.004965e+07     ko
2    1998-01-06  0  23.2502  23.5149  23.2044  23.3808  7.462927e+06     ko
3    1998-01-07  0  23.2255  23.3596  22.9855  23.3385  7.102218e+06     ko
4    1998-01-08  0  23.2255  23.6455  23.2044  23.5361  8.561481e+06     ko
...         ... ..      ...      ...      ...      ...           ...    ...
3921 2013-08-05  0  81.9700  82.4900  81.4600  82.3300  2.229676e+06    apa
3922 2013-08-06  0  82.2900  82.4900  81.6200  82.2900  3.076233e+06    apa
3923 2013-08-07  0  82.1200  82.5400  81.3500  81.9500  2.912846e+06    apa
3924 2013-08-08  0  82.0000  83.5100  81.9200  83.1700  2.099414e+06    apa
3925 2013-08-09  0  83.0400  83.5100  82.3800  83.2300  1.722495e+06    apa

[1808278 rows x 8 columns]


### Pivot to closing prices

In [3]:
close = df.pivot(index='date', columns='ticker', values='close')

In [4]:
close

ticker,a,aa,aapl,abbv,abc,abt,ace,acn,act,adbe,...,xl,xlnx,xom,xray,xrx,xyl,yhoo,yum,zion,zmh
date,,,,,,,,,,,,,,,,,,,,,
1998-01-02,NaN,13.35110,3.95098,NaN,6.50799,10.3555,22.9865,NaN,32.06,4.99041,...,40.6169,7.96400,21.6592,9.04339,30.5852,NaN,4.14437,6.14229,36.2098,NaN
1998-01-05,NaN,13.58530,3.89020,NaN,6.40419,10.4031,22.8365,NaN,33.63,5.05201,...,40.8146,7.80886,21.4182,8.77893,31.0476,NaN,3.92563,5.99731,36.9102,NaN
1998-01-06,NaN,13.28170,4.60502,NaN,6.28477,10.2311,23.0180,NaN,33.44,5.23685,...,40.6923,7.75715,20.6111,8.75765,30.7898,NaN,3.99250,5.75640,36.5122,NaN
1998-01-07,NaN,13.30420,4.24032,NaN,6.34839,10.2880,23.1389,NaN,32.69,5.15182,...,39.9766,7.42204,21.2435,8.70293,29.7423,NaN,3.98813,5.75640,35.4618,NaN
1998-01-08,NaN,12.75330,4.39107,NaN,6.38299,10.4507,22.7446,NaN,33.38,5.20604,...,39.4021,7.45928,20.8068,8.64518,29.1286,NaN,4.01562,5.53041,32.6286,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-08-05,45.87,7.97962,466.14200,44.67,59.01000,36.4600,90.9600,74.18,136.66,47.65000,...,31.6700,46.11000,90.9911,42.69000,10.0700,25.27,27.67000,74.78000,29.9200,83.94
2013-08-06,46.05,7.88000,462.30700,44.83,58.64000,36.0600,90.0400,73.21,135.59,47.38000,...,31.3600,45.56000,90.8322,42.54000,9.9700,25.42,27.29000,73.98000,29.3700,82.96
2013-08-07,46.30,7.78000,461.79100,44.59,58.77000,35.8900,89.8200,73.03,135.46,47.10000,...,31.2200,45.60000,90.8223,42.41000,9.8800,24.94,27.41000,73.90000,29.0800,82.61


### Check data for survivor bias

In [5]:
a =close.isnull().sum()
a = a[a>0]

In [6]:
for i in a.index:
    if close[i].tail(20).isnull().sum() >0:
        print(i)


cvh
hnz
pcs


In [7]:
print(close['cvh'].tail(70))

date
2013-05-02    49.81
2013-05-03    50.19
2013-05-06    50.11
2013-05-07      NaN
2013-05-08      NaN
              ...  
2013-08-05      NaN
2013-08-06      NaN
2013-08-07      NaN
2013-08-08      NaN
2013-08-09      NaN
Name: cvh, Length: 70, dtype: float64


Includes ticker with NaN values at tail. All is well.

### For each month-end observation period, rank the stocks by previous returns, from the highest to the lowest

In [8]:
monthly = close.resample('M').last()
returns = np.log(monthly) - np.log(monthly.shift(1))
prev_returns = returns.shift(1)
lookahead_returns = returns.shift(-1)

In [9]:
# rank the stocks by prev returns from highest to lowest
def rank_(prev_returns, n):
    top_prices = pd.DataFrame(0, index=prev_returns.index, columns=prev_returns.columns)
    for index, col in prev_returns.iterrows():
        top_prices.loc[index, col.nlargest(n).index] = 1
    return top_prices


### Select the top performing stocks for the long position, and the bottom performing stocks for the short position

In [10]:
# select top for long, bottom for short
long = rank_(prev_returns, 10)
short = rank_(-1 * prev_returns, 10) 


In [11]:
# calculate expected portfolio returns
port_ret = (long * lookahead_returns - short * lookahead_returns)/(10+10)
port_ret

ticker,a,aa,aapl,abbv,abc,abt,ace,acn,act,adbe,...,xl,xlnx,xom,xray,xrx,xyl,yhoo,yum,zion,zmh
date,,,,,,,,,,,,,,,,,,,,,
1998-01-31,NaN,0.0,0.000000,NaN,0.0,0.0,0.0,NaN,0.000000,0.0,...,0.0,0.00000,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,NaN
1998-02-28,NaN,0.0,0.000000,NaN,0.0,0.0,0.0,NaN,0.000000,0.0,...,0.0,0.00000,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,NaN
1998-03-31,NaN,0.0,-0.000456,NaN,0.0,0.0,0.0,NaN,0.000000,0.0,...,0.0,0.00000,0.0,0.0,0.0,NaN,0.000000,0.0,0.0,NaN
1998-04-30,NaN,0.0,0.000000,NaN,0.0,0.0,0.0,NaN,0.000000,0.0,...,0.0,0.00928,0.0,0.0,0.0,NaN,-0.004008,0.0,0.0,NaN
1998-05-31,NaN,0.0,0.000000,NaN,0.0,0.0,0.0,NaN,0.000000,0.0,...,0.0,0.00000,0.0,0.0,0.0,NaN,0.018121,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013-04-30,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2013-05-31,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.001256,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2013-06-30,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


## statistical tests

In [12]:
def t_test(returns):
    t_value = stats.ttest_1samp(returns,0.0)[0]
    p_value = stats.ttest_1samp(returns,0.0)[1]/2
    if p_value < 0.05:
        result = "Strategy might work. proceed further"
    else:
        result = "rethink strategy. There doesn't seem to be an alpha factor involved"
    return t_value, p_value, result


In [13]:
port_ret.T.sum().mean()

0.0060848424554046255

In [14]:
(np.exp(port_ret.T.sum().mean() * 12 ) -1 ) * 100

7.575001799545111

In [15]:
t, p , result= t_test(port_ret.T.sum())

In [16]:
print('t-value: ',t,'\np-value: ',p,'\nconclusion: ',result)

t-value:  1.5072186565119008 
p-value:  0.06672104990276045 
conclusion:  rethink strategy. There doesn't seem to be an alpha factor involved
